## Midterm project: Equity Portfolio Management

### Data Preparation

Download the historial daily data of the entire 2018 for the 10 stocks 

```python
universe = ['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN', 'META', 'NFLX', 'TSLA', 'ORCL', 'SAP']
```


You should have 10 csv files on your disk now. IBM.csv, MSFT.csv, etc. We call the 10 stocks "universe" which is the entire stock market you can trade.


![image.png](attachment:image.png)


### Retrieve the "Close" and "Adj Close" values for each stock

You will create a dataframe where there are 20 columns for the 10 stocks, each row is the "Close" and "Adj Close" prices for the 10 stocks on each day, in the order of the business days in 2018. Assume all buy/sell on the "Close" prices and there is no transaction cost.

### You start to manage 5 million dollars fund on Jan 02, 2018

You have a strategy to manage the fund.

1. On Jan 02 2018, you split the $\$5m$ into 5 $\$1m$, and use them to buy 5 stocks from the 10 stocks. For example, IBM close price was $\$154.25$. With $\$1m$, you can buy max 6482 shares with cost $\$999848.5$ with $\$151.5$‬ cash left. You decided to spend $\$1m$ on each of `['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN']` respectively and keep the rest cash into a zero-interest cash account. On Jan 02 2018, your mark to market value (MTM) is $\$5m$ if combining all stocks value and cash. Your holdings of stocks and cach account is your portfolio.

\begin{equation*}
\ MTM^t =  cash^t + \sum_{k=1}^5 Shares_k^t \times ClosePrice_k^t \
\end{equation*}


2. Your trading strategy is "5 days rebalancing of buying low". Here is how it works. You keep your portfolio unchanged until 5 days later on Jan 09 2018. Now you want to re-check the market and adjust your portfolio. You will compute the "Adj Close" price changes from Jan 02 to Jan 09, and find the 5 stocks whose "Adj Close" prices dropped the most in terms of percentage. You sell all current holdings on Jan 09 "Close" prices to convert your portfolio to all cash. Then immediately split your cash, including your cash account, to 5 equal parts to buy the 5 stocks that dropped the most from Jan 02 to Jan 09 on 'Adj Close' prices. You always buy the max shares of stock on the "Close" price and keep the rest cash in cash account. Now the portfolio should be different from 5 days ago. This operation is called "rebalancing".

    Keep in mind, the MTM will change every day, even when your portfolio holdings don't change, because the stock prices change.


3. Corporations generally issue stock dividends on some days. The total dividend you get on such a day is the stock dividend  times your shares if you have shares of this stock on the dividend day. If you buy shares on the dividend day, these bought shares are not qualified to get dividend. If you sell shares on the dividend day, the sold shares are qualified to get dividend. For example, on 2/8/2018, IBM issued \$1.5 dividend per share. In your cash account, you will automatically get

\begin{equation*}
\$1.5 \times \left(your\ IBM\ shares\ on\ 2/8/2018\right)
\end{equation*} 



4. 5 business days later on Jan 17 (Jan 15 was a holiday), you re-check the market and adjust your portfolio again. You will have a new portfolio on Jan 17.


5. If you run this strategy every 5 days all the way to Dec 31 2018, you will have a daily MTM. You expect the MTM on Dec 31 2018 should be higher than \$5m because you always buy the stocks that dropped the most, i.e., you always buy low.


6. Another strategy is "5 days rebalancing of buying high". You always buy the 5 stocks whose "Adj Close" prices surge the most in terms of percentage because you believe the trend will continue. Run the new strategy and see how the MTM will change.


7. You will create a "high tech index" which is simply the daily average of the 10 stocks "Close" prices. Compare your MTM series with the "high tech index" and plot their curves. To plot the two curves together, you may want to convert the series to daily percentage change with regard to Jan 02 2018.


8. Download the USD/JPY 2018 historical data at https://finance.yahoo.com/quote/JPY%3DX/history?period1=1514764800&period2=1546300800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true then use the "Close" column as the rate to convert your MTM series from USD to JPY. Plot the two MTM curves. You will need to convert to daily percentage change too.


9. The above two strategies both rebalance every 5 days. Try to change the days interval and find the optimal days interval that maximizes the MTM on 12/31/2018. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from warnings import filterwarnings
from matplotlib.lines import Line2D
filterwarnings('ignore')
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [2]:
IBM=pd.read_csv("IBM.csv")[["Date","Close","Adj Close"]]
MSFT=pd.read_csv("MSFT.csv")[["Date","Close","Adj Close"]]
GOOG=pd.read_csv("GOOG.csv")[["Date","Close","Adj Close"]]
AAPL=pd.read_csv("AAPL.csv")[["Date","Close","Adj Close"]]
AMZN=pd.read_csv("AMZN.csv")[["Date","Close","Adj Close"]]
META=pd.read_csv("META.csv")[["Date","Close","Adj Close"]]
NFLX=pd.read_csv("NFLX.csv")[["Date","Close","Adj Close"]]
TSLA=pd.read_csv("TSLA.csv")[["Date","Close","Adj Close"]]
ORCL=pd.read_csv("ORCL.csv")[["Date","Close","Adj Close"]]
SAP=pd.read_csv("SAP.csv")[["Date","Close","Adj Close"]]

In [3]:
universe = [IBM, MSFT, GOOG, AAPL, AMZN, META, NFLX, TSLA, ORCL, SAP]
for df in universe:
    df['Close Ratio']=[0 if i==0 else df.Close[i-1]/df.Close[i] for i in range(len(df.Close))]
    df['Adj Close Ratio']=[0 if i==0 else df['Adj Close'][i-1]/df['Adj Close'][i] for i in range(len(df['Adj Close']))]
    df['Ratio Difference']=round(np.abs(df['Close Ratio']-df['Adj Close Ratio']),4)
    df["Dividend"]=round(df['Ratio Difference']*df['Close'],6)
    df.drop(['Close Ratio','Adj Close Ratio','Ratio Difference'],inplace=True,axis=1)

In [4]:
def data(df):
    return df.head()

def rename(x,df):
    return df.rename(columns={'Close':x+'_Close','Adj Close':x+'_Adj_Close','Dividend':x+'_Dividend'},inplace=True)

names = ['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN', 'META', 'NFLX', 'TSLA', 'ORCL', 'SAP']

In [5]:
IBM.Date=pd.to_datetime(IBM.Date)
MSFT.Date=pd.to_datetime(IBM.Date)
GOOG.Date=pd.to_datetime(IBM.Date)
AAPL.Date=pd.to_datetime(IBM.Date)
AMZN.Date=pd.to_datetime(IBM.Date)
META.Date=pd.to_datetime(IBM.Date)
NFLX.Date=pd.to_datetime(IBM.Date)
TSLA.Date=pd.to_datetime(IBM.Date)
ORCL.Date=pd.to_datetime(IBM.Date)
SAP.Date=pd.to_datetime(IBM.Date)

In [6]:
for i,j in zip(universe,names):
    rename(j,i)

In [7]:
data(IBM)

,Date,IBM_Close,IBM_Adj_Close,IBM_Dividend
0,2018-01-02,147.466537,111.800133,0.0
1,2018-01-03,151.520081,114.873306,0.0
2,2018-01-04,154.588913,117.199913,0.0
3,2018-01-05,155.344162,117.772476,0.0
4,2018-01-08,156.281067,118.482796,0.0


In [8]:
data(MSFT)

,Date,MSFT_Close,MSFT_Adj_Close,MSFT_Dividend
0,2018-01-02,85.949997,80.391853,0.0
1,2018-01-03,86.349998,80.765961,0.0
2,2018-01-04,87.110001,81.476837,0.0
3,2018-01-05,88.190002,82.486992,0.0
4,2018-01-08,88.279999,82.571167,0.0


In [9]:
data(GOOG)

,Date,GOOG_Close,GOOG_Adj_Close,GOOG_Dividend
0,2018-01-02,53.250000,53.250000,0.0
1,2018-01-03,54.124001,54.124001,0.0
2,2018-01-04,54.320000,54.320000,0.0
3,2018-01-05,55.111500,55.111500,0.0
4,2018-01-08,55.347000,55.347000,0.0


In [10]:
data(AAPL)

,Date,AAPL_Close,AAPL_Adj_Close,AAPL_Dividend
0,2018-01-02,43.064999,40.776524,0.0
1,2018-01-03,43.057499,40.769421,0.0
2,2018-01-04,43.257500,40.958801,0.0
3,2018-01-05,43.750000,41.425121,0.0
4,2018-01-08,43.587502,41.271267,0.0


In [11]:
data(AMZN)

,Date,AMZN_Close,AMZN_Adj_Close,AMZN_Dividend
0,2018-01-02,59.450500,59.450500,0.0
1,2018-01-03,60.209999,60.209999,0.0
2,2018-01-04,60.479500,60.479500,0.0
3,2018-01-05,61.457001,61.457001,0.0
4,2018-01-08,62.343498,62.343498,0.0


In [12]:
data(META)

,Date,META_Close,META_Adj_Close,META_Dividend
0,2018-01-02,181.419998,181.419998,0.0
1,2018-01-03,184.669998,184.669998,0.0
2,2018-01-04,184.330002,184.330002,0.0
3,2018-01-05,186.850006,186.850006,0.0
4,2018-01-08,188.279999,188.279999,0.0


In [13]:
data(NFLX)

,Date,NFLX_Close,NFLX_Adj_Close,NFLX_Dividend
0,2018-01-02,201.070007,201.070007,0.0
1,2018-01-03,205.050003,205.050003,0.0
2,2018-01-04,205.630005,205.630005,0.0
3,2018-01-05,209.990005,209.990005,0.0
4,2018-01-08,212.050003,212.050003,0.0


In [14]:
data(TSLA)

,Date,TSLA_Close,TSLA_Adj_Close,TSLA_Dividend
0,2018-01-02,21.368668,21.368668,0.0
1,2018-01-03,21.150000,21.150000,0.0
2,2018-01-04,20.974667,20.974667,0.0
3,2018-01-05,21.105333,21.105333,0.0
4,2018-01-08,22.427334,22.427334,0.0


In [15]:
data(ORCL)

,Date,ORCL_Close,ORCL_Adj_Close,ORCL_Dividend
0,2018-01-02,46.630001,42.437176,0.0
1,2018-01-03,47.709999,43.420063,0.0
2,2018-01-04,48.180000,43.847794,0.0
3,2018-01-05,48.470001,44.111725,0.0
4,2018-01-08,48.980000,44.575863,0.0


In [16]:
data(SAP)

,Date,SAP_Close,SAP_Adj_Close,SAP_Dividend
0,2018-01-02,112.389999,101.210297,0.0
1,2018-01-03,113.309998,102.038773,0.0
2,2018-01-04,115.050003,103.605690,0.0
3,2018-01-05,116.330002,104.758369,0.0
4,2018-01-08,114.800003,103.380562,0.0


In [17]:
universe = [IBM, MSFT, GOOG, AAPL, AMZN, META, NFLX, TSLA, ORCL, SAP]
df=IBM.copy()
for i in universe[1:]:
    df=df.merge(i,how='outer',on='Date')

In [18]:
df.head()

,Date,IBM_Close,IBM_Adj_Close,IBM_Dividend,MSFT_Close,MSFT_Adj_Close,MSFT_Dividend,GOOG_Close,GOOG_Adj_Close,GOOG_Dividend,...,NFLX_Dividend,TSLA_Close,TSLA_Adj_Close,TSLA_Dividend,ORCL_Close,ORCL_Adj_Close,ORCL_Dividend,SAP_Close,SAP_Adj_Close,SAP_Dividend
0,2018-01-02,147.466537,111.800133,0.0,85.949997,80.391853,0.0,53.250000,53.250000,0.0,...,0.0,21.368668,21.368668,0.0,46.630001,42.437176,0.0,112.389999,101.210297,0.0
1,2018-01-03,151.520081,114.873306,0.0,86.349998,80.765961,0.0,54.124001,54.124001,0.0,...,0.0,21.150000,21.150000,0.0,47.709999,43.420063,0.0,113.309998,102.038773,0.0
2,2018-01-04,154.588913,117.199913,0.0,87.110001,81.476837,0.0,54.320000,54.320000,0.0,...,0.0,20.974667,20.974667,0.0,48.180000,43.847794,0.0,115.050003,103.605690,0.0
3,2018-01-05,155.344162,117.772476,0.0,88.190002,82.486992,0.0,55.111500,55.111500,0.0,...,0.0,21.105333,21.105333,0.0,48.470001,44.111725,0.0,116.330002,104.758369,0.0
4,2018-01-08,156.281067,118.482796,0.0,88.279999,82.571167,0.0,55.347000,55.347000,0.0,...,0.0,22.427334,22.427334,0.0,48.980000,44.575863,0.0,114.800003,103.380562,0.0


In [19]:
# mtm_score =[5000000]
# IBM_Stocks=[1000000//df['IBM_Close'].iloc[0]]*5
# MSFT_Stocks=[1000000//df['MSFT_Close'].iloc[0]]*5
# GOOG_Stocks=[1000000//df['GOOG_Close'].iloc[0]]*5
# AAPL_Stocks=[1000000//df['AAPL_Close'].iloc[0]]*5
# AMZN_Stocks=[1000000//df['AMZN_Close'].iloc[0]]*5
# META_Stocks=[0]*5
# NFLX_Stocks=[0]*5
# TSLA_Stocks=[0]*5
# ORCL_Stocks=[0]*5
# SAP_Stocks=[0]*5
# rest=[((1000000-(1000000//df['IBM_Close'].iloc[0]*df['IBM_Close'].iloc[0])))+
#      ((1000000-(1000000//df['MSFT_Close'].iloc[0]*df['MSFT_Close'].iloc[0]))) +
#      ((1000000-(1000000//df['GOOG_Close'].iloc[0]*df['GOOG_Close'].iloc[0])))+
#      ((1000000-(1000000//df['AAPL_Close'].iloc[0]*df['AAPL_Close'].iloc[0])))+
#      ((1000000-(1000000//df['AMZN_Close'].iloc[0]*df['AMZN_Close'].iloc[0])))+
#       0-0-0-0-0]*5
# Portfolio={'IBM':{'Stocks':[1000000//df['IBM_Close'].iloc[0]]*5,'Rest':[1000000-(1000000//df['IBM_Close'].iloc[0]*df['IBM_Close'].iloc[0])]*5},
#            'MSFT':{'Stocks':[1000000//df['MSFT_Close'].iloc[0]]*5,'Rest':[1000000-(1000000//df['MSFT_Close'].iloc[0]*df['MSFT_Close'].iloc[0])]*5},
#            'GOOG':{'Stocks':[1000000//df['GOOG_Close'].iloc[0]]*5,'Rest':[1000000-(1000000//df['GOOG_Close'].iloc[0]*df['GOOG_Close'].iloc[0])]*5},
#            'AAPL':{'Stocks':[1000000//df['AAPL_Close'].iloc[0]],'Rest':[1000000-(1000000//df['AAPL_Close'].iloc[0]*df['AAPL_Close'].iloc[0])]},
#            'AMZN':{'Stocks':[1000000//df['AMZN_Close'].iloc[0]],'Rest':[1000000-(1000000//df['AMZN_Close'].iloc[0]*df['AMZN_Close'].iloc[0])]},
#            'META':{'Stocks':[0],'Rest':[0]},
#            'NFLX':{'Stocks':[0],'Rest':[0]},
#            'TSLA':{'Stocks':[0],'Rest':[0]},
#            'ORCL':{'Stocks':[0],'Rest':[0]},
#            'SAP':{'Stocks':[0],'Rest':[0]}
names = ['IBM', 'MSFT', 'GOOG', 'AAPL', 'AMZN', 'META', 'NFLX', 'TSLA', 'ORCL', 'SAP']
# for i in names:
#     if df.index
#     a=[0]*5
#     df[i+'_Percent_Change']=
# percent_Change=[0]*5
# for i, row in df.iloc[1:,:].iterrows():
#     mtm_score+=[row['IBM_Close']*IBM_Stocks[i]+

    
#     if i%5==0:
        # print(row)

In [20]:
df

,Date,IBM_Close,IBM_Adj_Close,IBM_Dividend,MSFT_Close,MSFT_Adj_Close,MSFT_Dividend,GOOG_Close,GOOG_Adj_Close,GOOG_Dividend,...,NFLX_Dividend,TSLA_Close,TSLA_Adj_Close,TSLA_Dividend,ORCL_Close,ORCL_Adj_Close,ORCL_Dividend,SAP_Close,SAP_Adj_Close,SAP_Dividend
0,2018-01-02,147.466537,111.800133,0.0,85.949997,80.391853,0.0,53.250000,53.250000,0.0,...,0.0,21.368668,21.368668,0.0,46.630001,42.437176,0.0,112.389999,101.210297,0.0
1,2018-01-03,151.520081,114.873306,0.0,86.349998,80.765961,0.0,54.124001,54.124001,0.0,...,0.0,21.150000,21.150000,0.0,47.709999,43.420063,0.0,113.309998,102.038773,0.0
2,2018-01-04,154.588913,117.199913,0.0,87.110001,81.476837,0.0,54.320000,54.320000,0.0,...,0.0,20.974667,20.974667,0.0,48.180000,43.847794,0.0,115.050003,103.605690,0.0
3,2018-01-05,155.344162,117.772476,0.0,88.190002,82.486992,0.0,55.111500,55.111500,0.0,...,0.0,21.105333,21.105333,0.0,48.470001,44.111725,0.0,116.330002,104.758369,0.0
4,2018-01-08,156.281067,118.482796,0.0,88.279999,82.571167,0.0,55.347000,55.347000,0.0,...,0.0,22.427334,22.427334,0.0,48.980000,44.575863,0.0,114.800003,103.380562,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2018-12-21,106.061188,84.046196,0.0,98.230003,93.468269,0.0,48.977001,48.977001,0.0,...,0.0,21.318001,21.318001,0.0,44.000000,40.687527,0.0,96.139999,87.901962,0.0
246,2018-12-24,102.839386,81.493156,0.0,94.129997,89.566994,0.0,48.811001,48.811001,0.0,...,0.0,19.692667,19.692667,0.0,42.689999,39.476147,0.0,95.000000,86.859650,0.0
247,2018-12-26,106.491394,84.387115,0.0,100.559998,95.685303,0.0,51.973000,51.973000,0.0,...,0.0,21.739332,21.739332,0.0,44.590000,41.233105,0.0,98.379997,89.950020,0.0
248,2018-12-27,108.776291,86.197746,0.0,101.180000,96.275261,0.0,52.194000,52.194000,0.0,...,0.0,21.075333,21.075333,0.0,44.970001,41.584507,0.0,98.989998,90.507744,0.0


In [21]:
df['modd']=[0,1,2,3,4]*50

In [22]:
for j in names:
    df[j+'_Percent_change']=[round(((df[j+'_Adj_Close'][i]-df[j+'_Adj_Close'][i-5])/df[j+'_Adj_Close'][i-5])*100,3) 
                              if (df.index[i]%5==0 and i!=0) else 0 for i in range(len(df))]

In [26]:
df1=df[df.modd==0]

In [47]:
stocks=[]
l={'IBM','MSFT','GOOG','AAPL','AMZN'}
for i,row in df.iterrows():
    if i==0:
        stocks+=[{'IBM','MSFT','GOOG','AAPL','AMZN'}]
    elif i%5==0:
        d={'IBM':row['IBM_Percent_change'],
        'MSFT':row['MSFT_Percent_change'],
        'GOOG':row['GOOG_Percent_change'],
        'AAPL':row['AAPL_Percent_change'],
        'AMZN':row['AMZN_Percent_change'],
        'META':row['META_Percent_change'],
        'NFLX':row['NFLX_Percent_change'],
        'TSLA':row['TSLA_Percent_change'],
        'ORCL':row['ORCL_Percent_change'],
        'SAP':row['SAP_Percent_change']}
        l={i[0] for i in list(sorted(d.items(), key=lambda x: x[1]))[0:5]}
        stocks+=[l]
    else:
        stocks+=[l]
df['stocks']=stocks   

In [49]:
df.head(6)

,Date,IBM_Close,IBM_Adj_Close,IBM_Dividend,MSFT_Close,MSFT_Adj_Close,MSFT_Dividend,GOOG_Close,GOOG_Adj_Close,GOOG_Dividend,...,MSFT_Percent_change,GOOG_Percent_change,AAPL_Percent_change,AMZN_Percent_change,META_Percent_change,NFLX_Percent_change,TSLA_Percent_change,ORCL_Percent_change,SAP_Percent_change,stocks
0,2018-01-02,147.466537,111.800133,0.0,85.949997,80.391853,0.0,53.250000,53.250000,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"{IBM, MSFT, AMZN, AAPL, GOOG}"
1,2018-01-03,151.520081,114.873306,0.0,86.349998,80.765961,0.0,54.124001,54.124001,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"{IBM, MSFT, AMZN, AAPL, GOOG}"
2,2018-01-04,154.588913,117.199913,0.0,87.110001,81.476837,0.0,54.320000,54.320000,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"{IBM, MSFT, AMZN, AAPL, GOOG}"
3,2018-01-05,155.344162,117.772476,0.0,88.190002,82.486992,0.0,55.111500,55.111500,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"{IBM, MSFT, AMZN, AAPL, GOOG}"
4,2018-01-08,156.281067,118.482796,0.0,88.279999,82.571167,0.0,55.347000,55.347000,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"{IBM, MSFT, AMZN, AAPL, GOOG}"
5,2018-01-09,156.625244,118.743706,0.0,88.220001,82.515060,0.0,55.313000,55.313000,0.0,...,2.641,3.874,1.202,5.357,3.555,4.098,4.106,5.621,1.575,"{SAP, MSFT, META, AAPL, GOOG}"


In [ ]:
for i,row in df.iterrows():
    if  i==0:
        overall_mtm=[5000000]
        Portfolio={'IBM':{'Stocks':1000000//df['IBM_Close'].iloc[0],'Rest':1000000-(1000000//df['IBM_Close'].iloc[0]*df['IBM_Close'].iloc[0])},
           'MSFT':{'Stocks':1000000//df['MSFT_Close'].iloc[0],'Rest':1000000-(1000000//df['MSFT_Close'].iloc[0]*df['MSFT_Close'].iloc[0])},
           'GOOG':{'Stocks':1000000//df['GOOG_Close'].iloc[0],'Rest':1000000-(1000000//df['GOOG_Close'].iloc[0]*df['GOOG_Close'].iloc[0])},
           'AAPL':{'Stocks':1000000//df['AAPL_Close'].iloc[0],'Rest':1000000-(1000000//df['AAPL_Close'].iloc[0]*df['AAPL_Close'].iloc[0])},
           'AMZN':{'Stocks':1000000//df['AMZN_Close'].iloc[0],'Rest':1000000-(1000000//df['AMZN_Close'].iloc[0]*df['AMZN_Close'].iloc[0])},
           'META':{'Stocks':0,'Rest':0},
           'NFLX':{'Stocks':0,'Rest':0},
           'TSLA':{'Stocks':0,'Rest':0},
           'ORCL':{'Stocks':0,'Rest':0},
           'SAP':{'Stocks':0,'Rest':0}
            }
    elif i%5==0:
        mtm=(row[stocks[0]+'_Close']*Portfolio[stocks[0]].Stocks)+(row[stocks[1]+'_Close']*Portfolio[stocks[1]].Stocks)+
            (row[stocks[2]+'_Close']*Portfolio[stocks[2]].Stocks)+(row[stocks[3]+'_Close']*Portfolio[stocks[3]].Stocks)+
            (row[stocks[4]+'_Close']*Portfolio[stocks[4]].Stocks)+Portfolio[stocks[0]].Rest+Portfolio[stocks[1]].Rest+
            Portfolio[stocks[2]].Rest+Portfolio[stocks[3]].Rest+Portfolio[stocks[4]].Rest
        overall_mtm+=[mtm]
        Portfolio=